In [3]:
import pandas as pd
import numpy as np

# Tabela 1.1 - Distribuição da população, por cor ou raça, segundo Grandes Regiões e Unidades da Federação - Brasil - 2021 - 2012
def process_population_data(file_path, sheet_name):
    df_Distribuição_etária_da_população_por_cor_ou_raça_segundo_regioes = pd.read_excel(file_path, sheet_name = sheet_name)
    df_cleaned_r = df_Distribuição_etária_da_população_por_cor_ou_raça_segundo_regioes.copy()
    df_cleaned_r = df_cleaned_r.dropna(how = 'all')
    df_cleaned_r.columns = df_cleaned_r.iloc[2]

    rows_to_drop = [2, 3, 4, 5, 39]
    df_cleaned_r = df_cleaned_r.drop(rows_to_drop)
    df_cleaned_r.columns.values[1] = 'Quantity'
    df_cleaned_r.columns.values[0] = 'r'

    new_column_namesr = {
        'Branca': 'White',
        'Preta': 'Black',
        'Parda': 'Mixed',
        'Preta ou parda': 'Black or Mixed',
        'Quantity': 'Quantity1(10^3)'
    }

    df_cleaned_r = df_cleaned_r.rename(columns = new_column_namesr)



    Region = {
        'Norte': 'Norte', 'Rondônia': 'Norte', 'Acre': 'Norte', 'Amazonas': 'Norte', 'Roraima': 'Norte', 'Pará': 'Norte', 'Amapá': 'Norte', 'Tocantins': 'Norte',
        'Nordeste': 'Nordeste', 'Maranhão': 'Nordeste', 'Piauí': 'Nordeste', 'Ceará': 'Nordeste', 'Rio Grande do Norte': 'Nordeste', 'Paraíba': 'Nordeste', 'Pernambuco': 'Nordeste', 'Alagoas': 'Nordeste', 'Sergipe': 'Nordeste', 'Bahia': 'Nordeste',
        'Sudeste': 'Sudeste', 'Minas Gerais': 'Sudeste', 'Espírito Santo': 'Sudeste', 'Rio de Janeiro': 'Sudeste', 'São Paulo': 'Sudeste',
        'Sul': 'Sul', 'Paraná': 'Sul', 'Santa Catarina': 'Sul', 'Rio Grande do Sul': 'Sul',
        'Centro-Oeste': 'Centro-Oeste', 'Mato Grosso do Sul': 'Centro-Oeste', 'Mato Grosso': 'Centro-Oeste', 'Goiás': 'Centro-Oeste', 'Distrito Federal': 'Centro-Oeste'
    }


    df_cleaned_r['Region'] = df_cleaned_r['r'].apply(lambda x: Region[x] if x in Region else x)
    df_cleaned_r['State'] = df_cleaned_r['r'].apply(lambda x: x if x in Region else '')
    df_cleaned_r = df_cleaned_r.drop(columns = 'r')


    df_cleaned_r = df_cleaned_r.melt(id_vars=['Region', 'State', 'Quantity1(10^3)'], 
                    value_vars = ['White', 'Black', 'Mixed', 'Black or Mixed'],  
                    value_name ='Percentage')
    
    return df_cleaned_r

    

def combine_data(file_path, years):
    combined_df = pd.DataFrame()
    
    for year in years:
        sheet_name = str(year)
        df_year = process_population_data(file_path, sheet_name)
        df_year['Year'] = year
        combined_df = pd.concat([combined_df, df_year], ignore_index=True)
    
    return combined_df


file_path = 'C:\\Users\\Emerson Wu Li\\Desktop\\Files Para Social... Project\\Tabela 1.1 (Pop_Geo).xls'
years = [2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012]
df_Distribution_population_Region = combine_data(file_path, years) 



df_Distribution_population_Region['Quantity(10^3)'] = (df_Distribution_population_Region['Quantity1(10^3)'] * df_Distribution_population_Region['Percentage']) / 100
df_Distribution_population_Region = df_Distribution_population_Region.drop(columns = 'Quantity1(10^3)')
df_Distribution_population_Region = df_Distribution_population_Region.drop(columns = 'Percentage')

region_totals = df_Distribution_population_Region.groupby(['Region', 'Year'])['Quantity(10^3)'].sum().reset_index()
region_totals = region_totals.rename(columns={'Quantity(10^3)': 'Region_Total'})


df_Distribution_population_Region = pd.merge(df_Distribution_population_Region, region_totals, on=['Region', 'Year'])

df_Distribution_population_Region['Updated_Quantity'] = df_Distribution_population_Region['Region_Total'] * (df_Distribution_population_Region['Quantity(10^3)'] / df_Distribution_population_Region.groupby(['Region', 'Year'])['Quantity(10^3)'].transform('sum'))

df_Distribution_population_Region['Quantity(10^3)'] = df_Distribution_population_Region['Updated_Quantity']

df_Distribution_population_Region = df_Distribution_population_Region.drop(columns=['Region_Total', 'Updated_Quantity'])
df_Distribution_population_Region['Quantity(10^3)'] = df_Distribution_population_Region['Quantity(10^3)'].round().astype(int)
df_Distribution_population_Region = df_Distribution_population_Region.rename(columns = {4 : 'Race'})
df_Distribution_population_Region


,Region,State,Race,Year,Quantity(10^3)
0,Norte,Norte,White,2021,3279
1,Norte,Rondônia,White,2021,545
2,Norte,Acre,White,2021,138
3,Norte,Amazonas,White,2021,588
4,Norte,Roraima,White,2021,119
...,...,...,...,...,...
1275,Centro-Oeste,Centro-Oeste,Black or Mixed,2012,8762
1276,Centro-Oeste,Mato Grosso do Sul,Black or Mixed,2012,1268
1277,Centro-Oeste,Mato Grosso,Black or Mixed,2012,2000
1278,Centro-Oeste,Goiás,Black or Mixed,2012,3916


In [4]:
# Percentage of the population by race(Norte, Nordeste, etc...)

df_Distribution_population_Region_only_region = df_Distribution_population_Region.copy()

State_to_drop = ['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste']
df_Distribution_population_Region_only_region = df_Distribution_population_Region_only_region[df_Distribution_population_Region_only_region['State'].isin(State_to_drop)] 


race_that_want = ['White', 'Black', 'Mixed']
df_Distribution_population_Region_only_region = df_Distribution_population_Region_only_region[df_Distribution_population_Region_only_region['Race'].isin(race_that_want)]

group = df_Distribution_population_Region_only_region.groupby(['Region', 'Year'])['Quantity(10^3)'].transform('sum')
df_Distribution_population_Region_only_region['Total'] = group
df_Distribution_population_Region_only_region['Percentage'] = (df_Distribution_population_Region_only_region['Quantity(10^3)'] / df_Distribution_population_Region_only_region['Total']) * 100
df_Distribution_population_Region_only_region = df_Distribution_population_Region_only_region.drop(columns = ['State', 'Total'])
df_Distribution_population_Region_only_region



,Region,Race,Year,Quantity(10^3),Percentage
0,Norte,White,2021,3279,17.911182
8,Nordeste,White,2021,14190,24.857668
18,Sudeste,White,2021,45437,51.250324
23,Sul,White,2021,22813,75.604825
27,Centro-Oeste,White,2021,5746,34.955591
...,...,...,...,...,...
1216,Norte,Mixed,2012,11868,72.854512
1224,Nordeste,Mixed,2012,35366,64.886981
1234,Sudeste,Mixed,2012,30400,36.699020
1239,Sul,Mixed,2012,4736,16.832528


In [5]:
# Percentage of the population by race and by states

df_Distribution_population_Region_only_state = df_Distribution_population_Region.copy()

State_to_drop = ['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste']
df_Distribution_population_Region_only_state = df_Distribution_population_Region_only_state[~df_Distribution_population_Region_only_state['State'].isin(State_to_drop)] 


race_that_want = ['White', 'Black', 'Mixed']
df_Distribution_population_Region_only_state = df_Distribution_population_Region_only_state[df_Distribution_population_Region_only_state['Race'].isin(race_that_want)]

group = df_Distribution_population_Region_only_state.groupby(['Race', 'Year', 'Region'])['Quantity(10^3)'].transform('sum')
df_Distribution_population_Region_only_state['Percentage'] = (df_Distribution_population_Region_only_state['Quantity(10^3)'] / group) * 100
df_Distribution_population_Region_only_state

,Region,State,Race,Year,Quantity(10^3),Percentage
1,Norte,Rondônia,White,2021,545,16.610789
2,Norte,Acre,White,2021,139,4.236513
3,Norte,Amazonas,White,2021,589,17.951844
4,Norte,Roraima,White,2021,120,3.657422
5,Norte,Pará,White,2021,1452,44.254800
...,...,...,...,...,...,...
1242,Sul,Rio Grande do Sul,Mixed,2012,1243,26.240236
1244,Centro-Oeste,Mato Grosso do Sul,Mixed,2012,1110,14.188930
1245,Centro-Oeste,Mato Grosso,Mixed,2012,1771,22.638374
1246,Centro-Oeste,Goiás,Mixed,2012,3588,45.864758
